### 개요 
1. FastAPI 특징 및 주요 기능 간단 설명
2. llm 서버 주의사항?
3. langserve 있다는거 알려줌

## 2. fastAPI에 llm 호출 서빙

### 테스트 편의를 위하여 `get`만 사용하며, 필요 인자는 쿼리 매개변수로 받습니다.

In [ ]:
from fastapi import FastAPI
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from fastapi.responses import StreamingResponse

from dotenv import load_dotenv

load_dotenv()
# FastAPI 인스턴스 생성

app = FastAPI()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = llm | StrOutputParser()


@app.get("/invoke")
def sync_chat(message: str):
    response = chain.invoke(message)
    return response


@app.get("/ainvoke")
async def async_chat(message: str):
    response = await chain.ainvoke(message)
    return response


@app.get("/stream")
def sync_stream_chat(message: str):
    def event_stream():
        try:
            for chunk in chain.stream(message):
                if len(chunk) > 0:
                    yield f"{chunk}"
        except Exception as e:
            yield f"data: {str(e)}\n\n"

    return StreamingResponse(event_stream(), media_type="text/event-stream")


@app.get("/astream")
async def async_stream_chat(message: str):
    async def event_stream():
        try:
            async for chunk in chain.astream(message):
                if len(chunk) > 0:
                    yield f"{chunk}"
        except Exception as e:
            yield f"data: {str(e)}\n\n"

    return StreamingResponse(event_stream(), media_type="text/event-stream")

밑의 코드를 통해서 FastAPI를 실행시키고 당신의 브라우저에서 url로 접속하여 테스트 해보시오

ex: `http://127.0.0.1:8000/invoke?message=구글의 설립연도를 알려줘`

In [ ]:
import uvicorn
import nest_asyncio

nest_asyncio.apply()
uvicorn.run(app)

# 3. FastAPI VectorDB 호출

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import time
import asyncio

from dotenv import load_dotenv

DB_PATH = "./chroma_db"

load_dotenv()

app = FastAPI()

chroma = Chroma(
    collection_name="FastApiServing",
    persist_directory=DB_PATH,
    embedding_function=OpenAIEmbeddings(),
)

retriever = chroma.as_retriever(
    search_kwargs={
        "k": 4,
    }
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


@app.get("/invoke")
def sync_chat(message: str):
    response = chain.invoke(message)
    return response


@app.get("/add-content")
async def add_content(content: str, author: str):
    time.sleep(3)
    chroma.add_texts([content], [{"source": author}])
    return {"message": f"add-content: {content}"}


@app.get("/async-add-content")
async def async_add_content(content: str, author: str):
    await asyncio.sleep(3)
    await chroma.aadd_texts([content], [{"source": author}])
    return {"message": f"async-add-content: {content}"}


# 추가 예정
# get by ids()
# aget by ids()

# delete
# adelete

In [ ]:
chroma.add_texts(
    [
        "봉암 수원지",
    ],
    
    metadata=[{
            "depts1": "경남",
            "depts2": "진주시",
            "depts3": "봉암면",
            "imgUrl": "./imgs/boongamImg.jpeg",
        },
)

TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'

In [11]:
result = retriever.invoke("TF IDF 에 대하여 알려줘")

In [12]:
print(result)

[Document(id='2e79119d-f90f-49ca-a467-7a8d79e087e8', metadata={'source': '이동학'}, page_content='안녕하세용가리'), Document(id='39250963-d3fe-4b22-907f-f66f216b6e18', metadata={'source': '이동학'}, page_content='안녕하세용가리가리'), Document(id='33046dc2-49ef-4322-9203-4397f86b183f', metadata={'source': '이동학'}, page_content='안녕하세용가리가리'), Document(id='06fd1ee4-50cb-4324-adf2-41014403ef6c', metadata={'source': '이동학'}, page_content='안녕하세용가리가리')]


ex: `http://127.0.0.1:8000/async-add-content?content=Hello World&author=donghak`

In [ ]:
import uvicorn
import nest_asyncio

nest_asyncio.apply()
uvicorn.run(app)

## 4. FastAPI 서빙시 비동기 중요성

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from fastapi import FastAPI
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
import time
import asyncio


DB_PATH = "./chroma_db"

load_dotenv()

app = FastAPI()

chroma = Chroma(
    collection_name="FastApiServing",
    persist_directory=DB_PATH,
    embedding_function=OpenAIEmbeddings(),
)

retriever = chroma.as_retriever(
    search_kwargs={
        "k": 4,
    }
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


@app.get("/invoke")
def sync_chat(message: str):
    response = chain.invoke(message)
    return response


@app.get("/add-content")
async def add_content(content: str, author: str):
    time.sleep(3)
    # chroma.add_texts([content], [{"source": author}])
    return {"message": f"add-content: {content}"}


@app.get("/async-add-content")
async def async_add_content(content: str, author: str):
    await asyncio.sleep(3)
    # await chroma.aadd_texts([content], [{"source": author}])
    return {"message": f"async-add-content: {content}"}

비동기 비교 테스트 코드

In [ ]:
import asyncio

import httpx
import time


async def make_request(endpoint):
    url = f"http://127.0.0.1:8000/{endpoint}"

    # print(f"-----url: {url}------")

    async with httpx.AsyncClient(timeout=20) as client:
        response = await client.get(url)
        if response.status_code == 200:
            print(f"Response from {endpoint}: {response.json()}")
        else:
            print(f"Error from {endpoint}: {response.status_code}")
            print("Response content:", response.text)


async def main(endpoint):
    tasks = [
        make_request(f"{endpoint}?content=my name is donghak&author=donghak"),
        make_request("invoke?message=내이름을 하나만 말해봐"),
    ]
    await asyncio.gather(*tasks)


if __name__ == "__main__":

    # Test sync
    start_time = time.time()
    for _ in range(3):
        asyncio.run(main(endpoint="add-content"))
    end_time = time.time()
    syncResult = end_time - start_time

    #  test async
    start_time = time.time()
    for _ in range(3):
        asyncio.run(main(endpoint="async-add-content"))
    end_time = time.time()
    asyncResult = end_time - start_time

    print(f"Sync time: {syncResult}")
    print(f"Async time: {asyncResult}")